In [1]:
import os
os.chdir('../../')

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from tensorflow.keras.metrics import AUC
from keras.layers import Dense, Dropout
from keras.regularizers import l2
from keras.models import Sequential

In [4]:
df_train = pd.read_csv('data/sampling/over_sampling/smote/train_mean_nn_smote.csv')
df_test = pd.read_csv('data/feature_engineered/null_representative/test_null_mean_scaled.csv')

In [5]:
drop = [
    'EconomicSegment',
    'ContractRate_FM',
    'ContractRate_G1',
    'ContractRate_G2',
    'ContractRate_G3',
    'ContractRate_G4',
    'ContractRate_G5',
    'ContractRate_G6'
]

# ##----------------------------------------------------------------------------------
# df_train = mapping_columns_if_exist(df_train)
# df_test = mapping_columns_if_exist(df_test)

# def handle_unknown_label(train_series, test_series):
#     unique_labels = train_series.unique()
#     label_map = {label: idx for idx, label in enumerate(unique_labels)}
#     train_encoded = train_series.map(label_map)
#     test_encoded = test_series.map(lambda x: label_map.get(x, -1))
#     return train_encoded, test_encoded

# # 各カラムに対してカスタムエンコーディング関数を適用
# for col in combination_columns:
#     df_train[col], df_test[col] = handle_unknown_label(df_train[col], df_test[col])
# ##-----------------------------------------------------------------------------------

X = df_train.drop(columns=drop, axis=1)
y = df_train['ProdTaken']

test_feature = X.columns.drop('ProdTaken')
df_test = df_test[test_feature]

In [8]:
scores = []

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
for tr_idx, va_idx in skf.split(X, y):
    tr_x, va_x = X.iloc[tr_idx], X.iloc[va_idx]
    tr_y, va_y = y.iloc[tr_idx], y.iloc[va_idx]
    
    model = Sequential()
    model.add(Dense(256, activation='relu', input_shape=(X.shape[1],), kernel_regularizer=l2(0.01)))
    model.add(Dropout(0.2))
    model.add(Dense(128, activation='relu', kernel_regularizer=l2(0.001)))
    model.add(Dropout(0.2))
    # model.add(Dense(128, activation='relu', kernel_regularizer=l2(0.001)))
    # model.add(Dropout(0.2))
    # model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.001)))
    # model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[AUC()])
    
    batch_size = 128
    epochs = 50
    history = model.fit(tr_x, tr_y,
                        batch_size=batch_size, epochs=epochs,
                        verbose=1, validation_data=(va_x, va_y))
    
    pred = model.predict(va_x)
    score = roc_auc_score(va_y, pred)
    scores.append(score)

print('AUC : ', scores)

Epoch 1/50
38/38 [==============================] - 1s 14ms/step - loss: 15.0550 - auc_7: 0.5041 - val_loss: 2.4865 - val_auc_7: 0.6617
Epoch 2/50
38/38 [==============================] - 0s 6ms/step - loss: 4.3816 - auc_7: 0.5352 - val_loss: 1.3073 - val_auc_7: 0.4976
Epoch 3/50
38/38 [==============================] - 0s 5ms/step - loss: 1.5384 - auc_7: 0.5269 - val_loss: 0.9256 - val_auc_7: 0.4644
Epoch 4/50
38/38 [==============================] - 0s 5ms/step - loss: 0.9593 - auc_7: 0.5144 - val_loss: 0.8350 - val_auc_7: 0.5462
Epoch 5/50
38/38 [==============================] - 0s 5ms/step - loss: 0.8392 - auc_7: 0.5269 - val_loss: 0.8035 - val_auc_7: 0.5595
Epoch 6/50
38/38 [==============================] - 0s 5ms/step - loss: 0.8072 - auc_7: 0.5059 - val_loss: 0.7897 - val_auc_7: 0.5560
Epoch 7/50
38/38 [==============================] - 0s 5ms/step - loss: 0.7935 - auc_7: 0.5223 - val_loss: 0.7829 - val_auc_7: 0.5541
Epoch 8/50
38/38 [==============================] - 0s 5ms/s